## P2 Data Engineer Code Challenge

### 0. Setup

In [3]:
# Import libraries
import os
import urllib # Modules for working with URLs
import json # Modules that handles JSON data interchange format
import pandas as pd # Modules used to enhance data manipulation
import numpy as np # Modules for advanced mathematics

# Specify for exporting results
output_path = os.getcwd()

# Simplify the data pulling
def create_df(limit = "1000", contain = ""):
    # Setup query parameters
    url = "https://data.gov.au/data/api/3/action/datastore_search?"
    resource_id = "resource_id=809c77d8-fd68-4a2c-806f-c63d64e69842"
    query = url + resource_id
    if len(limit) != 0:
        query = query + "&limit=" + limit
    if len(contain) != 0:
        query = query + "&q=" + contain
    
    fileobj = urllib.request.urlopen(query)
    data = json.loads(fileobj.read())
    df = pd.DataFrame.from_dict(data['result']['records'])
    
    return df

### 1. How many passengers flew into and out of Australia for each airline in each month of 2019?


In [4]:
# Create dataframe
pass_2019 = create_df(limit = "10000", contain = "2019")

# Subset to required variables
pass_2019 = pass_2019[['Airline', 'Month_num', 'Passengers_In', 'Passengers_Out']]
# Convert 'Month_num' to int to correct ordering on plot
pass_2019 = pass_2019.astype({'Month_num': np.int})

Display a subset of the passengers coming in and out of Australia in 2019 based on airline and ordered by month.

In [37]:
# Outputs subset of results 
display(pass_2019.sort_values('Month_num'))

,Airline,Month_num,Passengers_In,Passengers_Out
0,Solomon Airlines,1,1690,1411
109,Federal Express Corporation,1,0,0
116,Qantas Airways,1,2394,2019
117,Qantas Airways,1,72093,65699
646,SriLankan Airlines,1,8550,7431
...,...,...,...,...
568,Korean Air,12,16894,18520
983,AirAsia X,12,46737,67575
544,Virgin Australia,12,507,580
526,China Airlines,12,5009,6269


                         Airline  Month_num Passengers_In Passengers_Out
0               Solomon Airlines          1          1690           1411
11                  Fiji Airways          1         26844          22497
15             Indonesia AirAsia          1         15508          12904
17    Thai Airways International          1         37513          35968
27                  Fiji Airways          2         14682          14383
...                          ...        ...           ...            ...
1350            Virgin Australia          2          6501           6089
1352            Tianjin Airlines          1          5731           3680
1353              Qantas Airways          1         34120          27555
1359             Air New Zealand          1        143252         132684
1363                     Jetstar          2         34319          35559

[223 rows x 4 columns]


The results look correct. Export this to a csv.

In [6]:
pass_2019.to_csv(output_path + "\passengers_in_out_aus_2019.csv")

### 2. In the last 6 months which port country was responsible for the most passengers coming into the country?

In [155]:
# Create dataframe
# Luckily for us, the last 6 months are the first 6 months of 2021, so we only
# need to filter by year being 2021
pass_2021 = create_df(limit = "10000", contain = "2021")

# Let's store the summary of passengers in by port country in a new dataframe
pass_2021_sum = pd.DataFrame(columns = ['Port_Country', 'Total_Passengers_In'])

# By port country, calculate total passengers in
for port in pass_2021['Port_Country'].unique():
    pass_nums = pass_2021.Passengers_In[pass_2021.Port_Country == port]
    pass_sum = pd.to_numeric(pass_nums).sum()
    pass_2021_sum = pass_2021_sum.append(
        pd.DataFrame([[port, pass_sum]], 
                     columns = ['Port_Country', 'Total_Passengers_In']), 
        ignore_index=False)
    
# Display port with most passengers in
max_pass = pass_2021_sum['Total_Passengers_In'].max()
display(pass_2021_sum.loc[pass_2021_sum['Total_Passengers_In'] == max_pass])

,Port_Country,Total_Passengers_In
0,New Zealand,46413


### 3. What is the 3 month rolling average of freight arriving to Australia for each airline throughout 2018?

In [171]:
# Create dataframe
freight_2018 = create_df(limit = "10000", contain = "2018")
freight_2018 = freight_2018[['Airline', 'Freight_In_(tonnes)', 'Month_num']]
freight_2018.columns = ['Airline', 'Freight_In', 'Month']
# Convert 'Month_num' to int
freight_2018 = freight_2018.astype({'Month': np.int})
freight_2018 = freight_2018.astype({'Freight_In': np.float})

# Create output dataframe
freight = pd.DataFrame({'Airline': freight_2018.Airline.unique()})
# Sort
freight = freight.sort_values('Airline').reset_index(drop=True)

# For each airline
for airline in freight.Airline:
    # For each month
    for month in range(1,13):
        # Calculate the rolling sum
        # Do this by first getting the range of months
        start = month-2
        end = month
        if start > 0:
            print(month)
            curr = freight_2018[(freight_2018['Airline'] == airline) & 
                                (freight_2018['Month'].between(start,end))]
            
            freight.loc[freight['Airline']]
#         curr = freight[freight['Month'].between(start,end)]
#         print(curr)

3
                          Airline  Freight_In  Month
168   Air Caledonie International       2.942      1
659   Air Caledonie International       4.184      3
1176  Air Caledonie International       4.546      2
4
                          Airline  Freight_In  Month
659   Air Caledonie International       4.184      3
1176  Air Caledonie International       4.546      2
1291  Air Caledonie International       8.042      4
5
                          Airline  Freight_In  Month
659   Air Caledonie International       4.184      3
1099  Air Caledonie International       2.332      5
1291  Air Caledonie International       8.042      4
6
                          Airline  Freight_In  Month
1043  Air Caledonie International       4.765      6
1099  Air Caledonie International       2.332      5
1291  Air Caledonie International       8.042      4
7
                          Airline  Freight_In  Month
329   Air Caledonie International       2.681      7
1043  Air Caledonie International   

1163  Air Niugini      34.505     10
3
          Airline  Freight_In  Month
120   Air Vanuatu       3.058      2
692   Air Vanuatu       5.485      3
1342  Air Vanuatu       2.534      1
4
         Airline  Freight_In  Month
120  Air Vanuatu       3.058      2
679  Air Vanuatu       5.139      4
692  Air Vanuatu       5.485      3
5
         Airline  Freight_In  Month
679  Air Vanuatu       5.139      4
692  Air Vanuatu       5.485      3
749  Air Vanuatu       4.137      5
6
          Airline  Freight_In  Month
679   Air Vanuatu       5.139      4
749   Air Vanuatu       4.137      5
1246  Air Vanuatu       3.313      6
7
          Airline  Freight_In  Month
749   Air Vanuatu       4.137      5
1246  Air Vanuatu       3.313      6
1251  Air Vanuatu       2.593      7
8
          Airline  Freight_In  Month
179   Air Vanuatu       3.083      8
774   Air Vanuatu       4.825      8
1246  Air Vanuatu       3.313      6
1251  Air Vanuatu       2.593      7
9
          Airline  Freight_In  M

1122  Beijing Capital Airlines     127.351      6
7
                       Airline  Freight_In  Month
706   Beijing Capital Airlines     189.476      7
789   Beijing Capital Airlines     244.582      5
1122  Beijing Capital Airlines     127.351      6
8
                       Airline  Freight_In  Month
559   Beijing Capital Airlines     179.822      8
706   Beijing Capital Airlines     189.476      7
1122  Beijing Capital Airlines     127.351      6
9
                      Airline  Freight_In  Month
559  Beijing Capital Airlines     179.822      8
706  Beijing Capital Airlines     189.476      7
968  Beijing Capital Airlines     214.746      9
10
                       Airline  Freight_In  Month
559   Beijing Capital Airlines     179.822      8
968   Beijing Capital Airlines     214.746      9
1234  Beijing Capital Airlines     220.878     10
11
                       Airline  Freight_In  Month
957   Beijing Capital Airlines     270.345     11
968   Beijing Capital Airlines     214.746

911  Delta Air Lines     266.860      3
4
             Airline  Freight_In  Month
161  Delta Air Lines     251.166      2
909  Delta Air Lines     270.317      4
911  Delta Air Lines     266.860      3
5
             Airline  Freight_In  Month
621  Delta Air Lines     277.592      5
909  Delta Air Lines     270.317      4
911  Delta Air Lines     266.860      3
6
              Airline  Freight_In  Month
621   Delta Air Lines     277.592      5
909   Delta Air Lines     270.317      4
1041  Delta Air Lines     325.591      6
7
              Airline  Freight_In  Month
310   Delta Air Lines     408.366      7
621   Delta Air Lines     277.592      5
1041  Delta Air Lines     325.591      6
8
              Airline  Freight_In  Month
195   Delta Air Lines     403.997      8
310   Delta Air Lines     408.366      7
1041  Delta Air Lines     325.591      6
9
             Airline  Freight_In  Month
195  Delta Air Lines     403.997      8
310  Delta Air Lines     408.366      7
764  Delta Air L

1268  Fiji Airways     253.096      5
7
           Airline  Freight_In  Month
398   Fiji Airways     349.286      6
1035  Fiji Airways     422.174      7
1268  Fiji Airways     253.096      5
8
           Airline  Freight_In  Month
398   Fiji Airways     349.286      6
702   Fiji Airways     424.748      8
1035  Fiji Airways     422.174      7
9
           Airline  Freight_In  Month
561   Fiji Airways     285.242      9
702   Fiji Airways     424.748      8
1035  Fiji Airways     422.174      7
10
          Airline  Freight_In  Month
350  Fiji Airways     346.801     10
561  Fiji Airways     285.242      9
702  Fiji Airways     424.748      8
11
           Airline  Freight_In  Month
350   Fiji Airways     346.801     10
561   Fiji Airways     285.242      9
1009  Fiji Airways     271.443     11
12
           Airline  Freight_In  Month
121   Fiji Airways     274.867     12
350   Fiji Airways     346.801     10
1009  Fiji Airways     271.443     11
3
               Airline  Freight_In  M

...

          Airline  Freight_In  Month
8     Malindo Air       0.000      3
219   Malindo Air       3.858      3
849   Malindo Air       0.200      4
854   Malindo Air       0.000      5
1102  Malindo Air       2.000      5
1190  Malindo Air       3.370      4
6
          Airline  Freight_In  Month
14    Malindo Air       0.225      6
849   Malindo Air       0.200      4
854   Malindo Air       0.000      5
891   Malindo Air       3.990      6
1102  Malindo Air       2.000      5
1190  Malindo Air       3.370      4
7
          Airline  Freight_In  Month
14    Malindo Air       0.225      6
846   Malindo Air       4.700      7
854   Malindo Air       0.000      5
891   Malindo Air       3.990      6
1102  Malindo Air       2.000      5
1180  Malindo Air       0.000      7
8
          Airline  Freight_In  Month
14    Malindo Air       0.225      6
846   Malindo Air       4.700      7
891   Malindo Air       3.990      6
934   Malindo Air       4.750      8
985   Malindo Air       1.559   

1324  Qantas Airways       1.730      5
6
             Airline  Freight_In  Month
69    Qantas Airways    1740.875      6
71    Qantas Airways     105.181      6
77    Qantas Airways     160.424      5
95    Qantas Airways    1477.469      4
100   Qantas Airways     121.372      4
131   Qantas Airways     488.900      6
150   Qantas Airways     721.979      4
160   Qantas Airways       2.295      6
211   Qantas Airways     423.714      5
231   Qantas Airways    1593.733      5
245   Qantas Airways       0.000      4
248   Qantas Airways      99.486      4
276   Qantas Airways    1019.436      5
356   Qantas Airways      89.731      6
364   Qantas Airways     221.051      4
368   Qantas Airways     153.334      4
384   Qantas Airways     184.048      6
405   Qantas Airways      20.943      5
466   Qantas Airways       0.000      6
478   Qantas Airways     135.841      6
524   Qantas Airways     378.023      4
530   Qantas Airways    1028.839      5
541   Qantas Airways       2.048      

1092  Sichuan Airlines      73.620     11
12
               Airline  Freight_In  Month
297   Sichuan Airlines      91.089     10
1092  Sichuan Airlines      73.620     11
1229  Sichuan Airlines      67.123     12
3
      Airline  Freight_In  Month
261  Silk Air      26.568      2
899  Silk Air      22.492      1
955  Silk Air      27.107      3
4
      Airline  Freight_In  Month
261  Silk Air      26.568      2
732  Silk Air      24.536      4
955  Silk Air      27.107      3
5
       Airline  Freight_In  Month
732   Silk Air      24.536      4
955   Silk Air      27.107      3
1130  Silk Air      23.223      5
6
       Airline  Freight_In  Month
9     Silk Air      22.401      6
732   Silk Air      24.536      4
1130  Silk Air      23.223      5
7
       Airline  Freight_In  Month
9     Silk Air      22.401      6
574   Silk Air      28.472      7
1130  Silk Air      23.223      5
8
      Airline  Freight_In  Month
9    Silk Air      22.401      6
574  Silk Air      28.472      7
829 

1039  Tasman Cargo Airlines     572.580     11
12
                    Airline  Freight_In  Month
542   Tasman Cargo Airlines     540.008     10
1039  Tasman Cargo Airlines     572.580     11
1250  Tasman Cargo Airlines     562.282     12
3
                         Airline  Freight_In  Month
543   Thai Airways International    1157.581      1
805   Thai Airways International    1253.077      2
1346  Thai Airways International    1533.803      3
4
                         Airline  Freight_In  Month
308   Thai Airways International    1307.978      4
805   Thai Airways International    1253.077      2
1346  Thai Airways International    1533.803      3
5
                         Airline  Freight_In  Month
308   Thai Airways International    1307.978      4
575   Thai Airways International    1495.886      5
1346  Thai Airways International    1533.803      3
6
                        Airline  Freight_In  Month
28   Thai Airways International    1434.936      6
308  Thai Airways Internatio

Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
4
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
5
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
6
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
7
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
8
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
9
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
10
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
11
Empty DataFrame
Columns: [Airline, Freight_In, Month]
Index: []
12
          Airline  Freight_In  Month
249  Virgin Samoa        0.42     12
3
              Airline  Freight_In  Month
113   Xiamen Airlines     230.289      2
571   Xiamen Airlines     421.725      3
1126  Xiamen Airlines     405.411      1
4
              Airline  Freight_In  Month
113   Xiamen Airlines     230.289      2
571   Xiamen Airlines     421.725      3
1169  Xiamen Airlines     493.076     